In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load 
file_to_load = "/Users/arunima_menon/Desktop/04-Pandas/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

In [4]:
# Get a unique number of players by 'SN' from the file
unique_users = purchase_data['SN'].nunique()
total_players = pd.DataFrame({'Total Players': [unique_users]})
total_players

,Total Players
0,576


In [5]:
# From the file, calculate metrics for Purchasing Analysis
unique_items = purchase_data['Item Name'].nunique()
avg_price = purchase_data['Price'].mean()
num_purchases = purchase_data['Item Name'].count()
total_revenue = purchase_data['Price'].sum()

# Create dataframe with metrics
purchasing_analysis_summary = pd.DataFrame(
        {'Number of Unique Items': [unique_items],
         'Average Price': [avg_price],
         'Number of Purchases': [num_purchases],
         'Total Revenue': [total_revenue],
        } )

# format columns
purchasing_analysis_summary['Average Price'] = purchasing_analysis_summary['Average Price'].map("$ {:,.2f}".format)
purchasing_analysis_summary['Total Revenue'] = purchasing_analysis_summary['Total Revenue'].map("$ {:,.2f}".format)


purchasing_analysis_summary[['Number of Unique Items','Average Price', 'Number of Purchases','Total Revenue']]

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$ 3.05,780,"$ 2,379.77"


In [6]:
# Group by Gender and get Percentage of Players based on Gender Totals
gender_groupby = purchase_data.groupby('Gender')['SN'].nunique().reset_index()
gender_groupby['Percentage of Players'] = 100 * gender_groupby['SN']/gender_groupby['SN'].sum()
gender_summary = gender_groupby[['Gender', 'Percentage of Players','SN' ]].sort_values(['Percentage of Players'],ascending = False)
gender_summary = gender_summary.reset_index(drop=True)

# format columns
gender_summary['Percentage of Players'] = gender_summary['Percentage of Players'].map("{:,.2f}".format)

# set Index to Gender
gender_demos_summary = gender_summary.set_index('Gender')
gender_demos_summary = gender_demos_summary.rename(columns = {'SN': 'Total Count'}) 
gender_demos_summary

,Percentage of Players,Total Count
Gender,,
Male,84.03,484
Female,14.06,81
Other / Non-Disclosed,1.91,11


In [7]:
# Group by Gender and get summary of data
gender_demos = purchase_data.groupby('Gender')["Price"].agg(['sum', 'mean','count']).sort_values(['sum'],ascending = False).reset_index()

# Calculate normalized values
gender_normal_f= gender_demos[['sum']].iloc[1,0]/gender_summary[['SN']].iloc[1,0]
gender_normal_m= gender_demos[['sum']].iloc[0,0]/gender_summary[['SN']].iloc[0,0]
gender_normal_o = gender_demos[['sum']].iloc[2,0]/gender_summary[['SN']].iloc[2,0]
# Add Normalized Totals column to summary table
gender_summary['Normalized Totals'] = [gender_normal_m, gender_normal_f, gender_normal_o]

# Merge Table with gender summary information and gender demographic information.  Rename Columns
merge_gender = pd.merge(gender_summary, gender_demos, on= 'Gender')
merge_gender_df = merge_gender.rename(columns = {'count': 'Purchase Count', 
                                               'mean' : 'Average Purchase Price', 
                                               'sum' : 'Total Purchase Value' })

# format columns
merge_gender_df['Average Purchase Price'] = merge_gender_df['Average Purchase Price'].map("$ {:,.2f}".format)
merge_gender_df['Total Purchase Value'] = merge_gender_df['Total Purchase Value'].map("$ {:,.2f}".format)
merge_gender_df['Normalized Totals'] = merge_gender_df['Normalized Totals'].map("$ {:,.2f}".format)

# set Index to Gender
purchasing_analysis_gender = merge_gender_df.set_index('Gender')
purchasing_analysis_gender[['Purchase Count', 'Average Purchase Price', 'Total Purchase Value', 'Normalized Totals']]

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,,
Male,652,$ 3.02,"$ 1,967.64",$ 4.07
Female,113,$ 3.20,$ 361.94,$ 4.47
Other / Non-Disclosed,15,$ 3.35,$ 50.19,$ 4.56


In [8]:
# Create the bins for Age Buckets
age_bins = [0, 9, 14, 19, 24, 29, 34, 39, 44]

# Create the labels for bins
age_groups = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+' ]

# Add column of bins based on Age
purchase_data["Age Groups"] = pd.cut(purchase_data["Age"],age_bins, labels=age_groups)

# Calculate total count and percentage of those counts by age groups
age_groupby = purchase_data.groupby('Age Groups')['SN'].nunique().reset_index()
age_groupby['Percentage of Players'] = 100 * age_groupby['SN']/gender_groupby['SN'].sum()
age_summary = age_groupby[['Age Groups', 'Percentage of Players','SN' ]].sort_values(['Age Groups'])
age_summary = age_summary.reset_index(drop=True)

# format columns
age_summary['Percentage of Players'] = age_summary['Percentage of Players'].map("{:,.2f}".format)

# set Indext to Age Groups
age_demos_summary = age_summary.set_index('Age Groups')
age_demos_summary = age_demos_summary.rename(columns = {'SN': 'Total Count'}) 
age_demos_summary

,Percentage of Players,Total Count
Age Groups,,
<10,2.95,17
10-14,3.82,22
15-19,18.58,107
20-24,44.79,258
25-29,13.37,77
30-34,9.03,52
35-39,5.38,31
40+,1.91,11


In [10]:
# get list of top 5 spenders
top5_spenders = purchase_data.groupby('SN')['Price'].sum().sort_values(ascending = False).nlargest(5).reset_index()
top5_spenders_SN = top5_spenders[['SN']]

# merge top 5 spenders with original data set to obtain summary data 
merge_top5_spenders = pd.merge(top5_spenders_SN, purchase_data, on = 'SN', how = 'left')
merge_top5_spenders = merge_top5_spenders.groupby('SN')['Price'].agg(['sum', 'mean','count']).sort_values(['sum'], ascending= False)

# rename columns
top_spenders = merge_top5_spenders.rename(columns = {'count': 'Purchase Count', 'mean' : 'Average Purchase Price', 
                                                      'sum' : 'Total Purchase Value' })
# format columns
top_spenders['Average Purchase Price'] = top_spenders['Average Purchase Price'].map("$ {:,.2f}".format)
top_spenders['Total Purchase Value'] = top_spenders['Total Purchase Value'].map("$ {:,.2f}".format)

top_spenders[['Purchase Count', 'Average Purchase Price', 'Total Purchase Value']]

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$ 3.79,$ 18.96
Idastidru52,4,$ 3.86,$ 15.45
Chamjask73,3,$ 4.61,$ 13.83
Iral74,4,$ 3.40,$ 13.62
Iskadarya95,3,$ 4.37,$ 13.10


In [11]:
# get list of top 5 most profitable items by sum
items_bysum = purchase_data.groupby(['Item ID','Item Name'])['Price'].agg(['count','sum','mean']).sort_values(['sum'], ascending= False).reset_index()
top5sum_items = items_bysum[:5]

# rename columns
profitable_items = top5sum_items.rename(columns = {'count': 'Purchase Count', 'mean' : 'Item Price', 
                                                    'sum' : 'Total Purchase Value' })
#format columns
profitable_items['Item Price'] = profitable_items['Item Price'].map("$ {:,.2f}".format)
profitable_items['Total Purchase Value'] = profitable_items['Total Purchase Value'].map("$ {:,.2f}".format)

# set Index to Item ID and Item Name
profitable_items_df = profitable_items.set_index(['Item ID', 'Item Name'])
profitable_items_df[['Purchase Count', 'Item Price', 'Total Purchase Value']]

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$ 4.23,$ 50.76
82,Nirvana,9,$ 4.90,$ 44.10
145,Fiery Glass Crusader,9,$ 4.58,$ 41.22
92,Final Critic,8,$ 4.88,$ 39.04
103,Singed Scalpel,8,$ 4.35,$ 34.80
